In [1]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

In [2]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH04-Models")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH04-Models


In [3]:
import os
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate

# 프롬프트 템플릿을 사용하여 질문을 생성합니다.
prompt = PromptTemplate.from_template("{fruit}의 색상이 무엇입니까?")

In [4]:
# 직렬화가 가능한지 체크합니다.
print(f"ChatOpenAI: {ChatOpenAI.is_lc_serializable()}")

ChatOpenAI: True


In [5]:
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

# 직렬화가 가능한지 체크합니다.
print(f"ChatOpenAI: {llm.is_lc_serializable()}")

ChatOpenAI: True


In [6]:
# 체인을 생성합니다.
chain = prompt | llm

# 직렬화가 가능한지 체크합니다.
chain.is_lc_serializable()

True

### 체인(Chain) 직렬화(dumps, dumpd)

In [7]:
from langchain_core.load import dumpd, dumps

dumpd_chain = dumpd(chain)
dumpd_chain

{'lc': 1,
 'type': 'constructor',
 'id': ['langchain', 'schema', 'runnable', 'RunnableSequence'],
 'kwargs': {'first': {'lc': 1,
   'type': 'constructor',
   'id': ['langchain', 'prompts', 'prompt', 'PromptTemplate'],
   'kwargs': {'input_variables': ['fruit'],
    'template': '{fruit}의 색상이 무엇입니까?',
    'template_format': 'f-string'},
   'name': 'PromptTemplate',
   'graph': {'nodes': [{'id': 0, 'type': 'schema', 'data': 'PromptInput'},
     {'id': 1,
      'type': 'runnable',
      'data': {'id': ['langchain', 'prompts', 'prompt', 'PromptTemplate'],
       'name': 'PromptTemplate'}},
     {'id': 2, 'type': 'schema', 'data': 'PromptTemplateOutput'}],
    'edges': [{'source': 0, 'target': 1}, {'source': 1, 'target': 2}]}},
  'last': {'lc': 1,
   'type': 'constructor',
   'id': ['langchain', 'chat_models', 'openai', 'ChatOpenAI'],
   'kwargs': {'model_name': 'gpt-3.5-turbo',
    'temperature': 0.0,
    'openai_api_key': {'lc': 1, 'type': 'secret', 'id': ['OPENAI_API_KEY']},
    'openai_p

In [8]:
# 직렬화된 체인의 타입을 확인합니다.
type(dumpd_chain)

dict

In [9]:
# dumps 함수를 사용하여 직렬화된 체인을 확인합니다.
dumps_chain = dumps(chain)
dumps_chain

'{"lc": 1, "type": "constructor", "id": ["langchain", "schema", "runnable", "RunnableSequence"], "kwargs": {"first": {"lc": 1, "type": "constructor", "id": ["langchain", "prompts", "prompt", "PromptTemplate"], "kwargs": {"input_variables": ["fruit"], "template": "{fruit}\\uc758 \\uc0c9\\uc0c1\\uc774 \\ubb34\\uc5c7\\uc785\\ub2c8\\uae4c?", "template_format": "f-string"}, "name": "PromptTemplate", "graph": {"nodes": [{"id": 0, "type": "schema", "data": "PromptInput"}, {"id": 1, "type": "runnable", "data": {"id": ["langchain", "prompts", "prompt", "PromptTemplate"], "name": "PromptTemplate"}}, {"id": 2, "type": "schema", "data": "PromptTemplateOutput"}], "edges": [{"source": 0, "target": 1}, {"source": 1, "target": 2}]}}, "last": {"lc": 1, "type": "constructor", "id": ["langchain", "chat_models", "openai", "ChatOpenAI"], "kwargs": {"model_name": "gpt-3.5-turbo", "temperature": 0.0, "openai_api_key": {"lc": 1, "type": "secret", "id": ["OPENAI_API_KEY"]}, "openai_proxy": "", "max_retries": 2

In [10]:
# 직렬화된 체인의 타입을 확인합니다.
type(dumps_chain)

str

### Pickle 파일

In [11]:
import pickle

# fuit_chain.pkl 파일로 직렬화된 체인을 저장합니다.
with open("fruit_chain.pkl", "wb") as f:
    pickle.dump(dumpd_chain, f)

In [12]:
import json

with open("fruit_chain.json", "w") as fp:
    json.dump(dumpd_chain, fp)

### load: 저장한 모델 불러오기

In [13]:
import pickle

# pickle 파일을 로드합니다.
with open("fruit_chain.pkl", "rb") as f:
    loaded_chain = pickle.load(f)

In [14]:
from langchain_core.load import load

# 체인을 로드합니다.
chain_from_file = load(loaded_chain)

# 체인을 실행합니다.
print(chain_from_file.invoke({"fruit": "사과"}))

c:\Users\UserK\miniforge3\envs\rag\lib\site-packages\langchain_core\_api\beta_decorator.py:87: LangChainBetaWarning: The function `load` is in beta. It is actively being worked on, so the API may change.
  warn_beta(


content='사과의 색상은 주로 빨간색이지만, 녹색, 노란색, 주황색 등 다양한 색상의 사과도 있습니다.' response_metadata={'token_usage': {'completion_tokens': 51, 'prompt_tokens': 24, 'total_tokens': 75}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-297ad430-d6d4-47da-9600-efd346f3ab09-0' usage_metadata={'input_tokens': 24, 'output_tokens': 51, 'total_tokens': 75}


In [15]:
from langchain_core.load import load, loads

load_chain = load(
    loaded_chain, secrets_map={"OPENAI_API_KEY": os.environ["OPENAI_API_KEY"]}
)

# 불러온 체인이 정상 동작하는지 확인합니다.
load_chain.invoke({"fruit": "사과"})

AIMessage(content='사과의 색상은 주로 빨간색이지만, 녹색, 노란색, 주황색 등 다양한 색상의 사과도 있습니다.', response_metadata={'token_usage': {'completion_tokens': 51, 'prompt_tokens': 24, 'total_tokens': 75}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-89df4903-328a-4e00-941f-d8cff7c54272-0', usage_metadata={'input_tokens': 24, 'output_tokens': 51, 'total_tokens': 75})

In [16]:
with open("fruit_chain.json", "r") as fp:
    loaded_from_json_chain = json.load(fp)
    loads_chain = load(loaded_from_json_chain)

In [17]:
# 불러온 체인이 정상 동작하는지 확인합니다.
loads_chain.invoke({"fruit": "사과"})

AIMessage(content='사과의 색상은 주로 빨간색이지만, 녹색, 노란색, 주황색 등 다양한 색상의 사과도 있습니다.', response_metadata={'token_usage': {'completion_tokens': 51, 'prompt_tokens': 24, 'total_tokens': 75}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-df3afa79-c4dc-48c1-a8a4-644df73d0286-0', usage_metadata={'input_tokens': 24, 'output_tokens': 51, 'total_tokens': 75})